In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, roc_auc_score, make_scorer

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from functools import reduce
from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 

%cd ../

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303


In [2]:
df = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
test_ids = deepcopy(test_data.id)

In [3]:
df['item_hash'] = df['name_mfi'].apply(hash) ^ df.weight_mfi.apply(lambda x: hash(str(x))) ^ df.weight.apply(lambda x: hash(str(x))) ^ \
    df.mailtype.apply(lambda x: hash(str(x))) ^ df.transport_pay.apply(lambda x: hash(str(x)))

test_data['item_hash'] = test_data['item_hash'] = test_data['name_mfi'].apply(hash) ^ test_data.weight_mfi.apply(lambda x: hash(str(x))) ^ test_data.weight.apply(lambda x: hash(str(x))) ^ \
    test_data.mailtype.apply(lambda x: hash(str(x))) ^ test_data.transport_pay.apply(lambda x: hash(str(x)))

In [4]:
hash_counter = Counter(pd.concat([df.item_hash, test_data.item_hash], ignore_index=True))

In [5]:
X = df.drop(columns=["label"])
y = df.label
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [6]:
top_operators = [obj for obj, count in Counter(test_data['index_oper'].replace(' ', '0').astype(float).astype(int)).most_common()][:10]

In [7]:
useless = []
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]
cat_features = list(set(cat_features) - set(useless))

def prepare_data(df):
    oper_type, oper_attr = zip(*df['oper_type + oper_attr'].map(lambda x: x.split('_')))
    
    # df['is_negative_oper_attr'] = pd.Series(oper_attr) == '-1'
    df['specialization_ratio'] = df.total_qty_over_index_and_type / df.total_qty_over_index.clip(lower=1)
    df['total_over_person'] = df.total_qty_over_index / df.dist_qty_oper_login_1.clip(lower=1)
    df['unknown_oper_ratio'] = df.total_qty_oper_login_0 / df.total_qty_oper_login_1.clip(lower=1)
    
    df['same_hash'] = df.item_hash.apply(lambda h: hash_counter[h])
    
    for operator in top_operators:
        df[f'is_{operator}_operator'] = df['index_oper'].replace(' ', '0').astype(float).astype(int) == operator
    
    replace_zeros = ['priority', 'is_privatecategory', 'is_in_yandex', 'mailtype', 'directctg']
    for column in replace_zeros:
        df[column][df[column] == 0] = df[column].mode()[0]
        df[column][df[column] == '0'] = df[column].mode()[0]
    
    df.priority = 1 * (df.priority == 7503.) + 2 * (df.priority == 7504.) + 3 * (df.priority == 7506.)
    
    df.index_oper = df.index_oper.replace(' ', '0').astype(float).astype(int)
    
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
    df['name_mfi_count'] = df.name_mfi.apply(lambda name: 0 if name == '0' else len(name.split(',')))
    
    df.is_return = df.is_return.apply(lambda text: text == 'Y')
        
    df.drop(columns=["oper_type + oper_attr"], inplace=True)
    df.drop(columns=["id", "name_mfi", "item_hash"], inplace=True)
    df.drop(columns=['mailrank'], inplace=True)
    df.drop(columns=useless, inplace=True, errors='ignore')
    
    return df

In [8]:
X_train = prepare_data(X_train)
X_val = prepare_data(X_val)
test_df = prepare_data(test_data)

In [9]:
X_train.to_csv('data/X_train_prepared.csv', index=False)
X_val.to_csv('data/X_val_prepared.csv', index=False)
y_train.to_csv('data/y_train_prepared.csv', index=False)
y_val.to_csv('data/y_val_prepared.csv', index=False)
test_df.to_csv('data/test_df_prepared.csv', index=False)
y_train.to_csv('data/y_train_prepared.csv', index=False)
y_val.to_csv('data/y_val_prepared.csv', index=False)

In [2]:
useless = []
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]
cat_features = list(set(cat_features) - set(useless))

cat_features = list(set(cat_features) - set(useless))
def to_cat_features(df):
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
X_train = pd.read_csv('data/X_train_prepared.csv')
y_train = pd.read_csv('data/y_train_prepared.csv')
X_val = pd.read_csv('data/X_val_prepared.csv')
y_val = pd.read_csv('data/y_val_prepared.csv')
test_df = pd.read_csv('data/test_df_prepared.csv')
test_ids = pd.read_csv('data/test_ids.csv')


to_cat_features(X_train)
to_cat_features(X_val)
to_cat_features(test_df)

In [3]:
model = CatBoostClassifier(random_state=0, max_depth=8, verbose=10, iterations=100, auto_class_weights="Balanced", cat_features=cat_features)

In [4]:
model.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.4007231	total: 1.86s	remaining: 3m 4s
10:	learn: 0.2555972	total: 19.1s	remaining: 2m 34s
20:	learn: 0.2474162	total: 36.1s	remaining: 2m 15s
30:	learn: 0.2438903	total: 53.5s	remaining: 1m 58s
40:	learn: 0.2409905	total: 1m 10s	remaining: 1m 41s
50:	learn: 0.2387495	total: 1m 27s	remaining: 1m 24s
60:	learn: 0.2369813	total: 1m 45s	remaining: 1m 7s
70:	learn: 0.2353248	total: 2m 2s	remaining: 50.1s
80:	learn: 0.2341042	total: 2m 19s	remaining: 32.7s
90:	learn: 0.2326741	total: 2m 36s	remaining: 15.5s
99:	learn: 0.2317223	total: 2m 52s	remaining: 0us


In [5]:
def score_model(model, X, y, threshold=0.5):  # 0.225 is the best for now
    pred = model.predict_proba(X)[:, 1] > threshold
    recall = recall_score(y, pred, average="macro")
    roc_auc = roc_auc_score(y,  pred, multi_class='ovo')
    score = 0.1 * recall + 0.9 * roc_auc
    return score

In [6]:
score_model(model, X_val, y_val, 0.35)

0.8923339688020783

In [7]:
score_model(model, X_train, y_train, 0.35) 

0.8966559886650828

In [8]:
sorted(list(zip(X_train.columns, model.feature_importances_)), key = lambda x: -x[1])

[('total_over_person', 23.03528617304492),
 ('specialization_ratio', 11.636410578186144),
 ('mailctg', 9.048432378373766),
 ('unknown_oper_ratio', 8.645245750744765),
 ('total_qty_over_index_and_type', 6.559096451066599),
 ('index_oper', 5.682813941654337),
 ('same_hash', 5.020983770979183),
 ('total_qty_oper_login_0', 4.821415396477959),
 ('transport_pay', 4.568592256740448),
 ('dist_qty_oper_login_1', 3.7171300924680954),
 ('weight_mfi', 3.3867163824978763),
 ('type', 2.613996983264388),
 ('total_qty_over_index', 2.5709130419609725),
 ('total_qty_oper_login_1', 2.144632912965941),
 ('price_mfi', 1.8361312619190995),
 ('weight', 1.3288023140740526),
 ('is_wrong_phone_number', 1.042432370242529),
 ('name_mfi_count', 1.0345835192214325),
 ('is_wrong_rcpn_name', 0.4565989390708098),
 ('priority', 0.4429076339424519),
 ('directctg', 0.20276511132961408),
 ('is_102976_operator', 0.09071092537307596),
 ('class', 0.07799415523402993),
 ('is_return', 0.014708309804661136),
 ('is_wrong_address

In [9]:
test_preds = (model.predict_proba(test_df)[:, 1] > 0.35).astype(int)
submission = pd.DataFrame({'id': test_ids.id, 'label': test_preds})
submission.to_csv("submission.csv", index=False)

In [5]:
unrounded_sub = pd.read_csv('submission.csv') 
sub = unrounded_sub
sub.label = (unrounded_sub.label > 0.225).astype(int) 
sub.to_csv('submission_rounded.csv', index=False)

In [6]:
sub.label.mean()

0.23711925